## RAG Setup Using Llama 4 Maverick & LangChain

### Install & import the required dependencies

In [7]:
!pip install --quiet langchain langchain-community langchain-openai pypdf pdf2image pdfminer.six singlestoredb tiktoken unstructured==0.10.14

In [ ]:
!pip install -U langchain-openai --quiet

In [8]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import SingleStoreDB
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

### Set your OpenAI API key

In [9]:
os.environ["OPENAI_API_KEY"] = "Add your OpenAI API Key"

### Load the PDF document

In [10]:
pdf_path = "https://mospi.gov.in/sites/default/files/publication_reports/EnviStats_India_2024.pdf"  # Replace with the path to your PDF file
loader = PyPDFLoader(pdf_path)
documents = loader.load()

### Split the document into chunks

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

### Initialize the OpenAI embeddings

In [14]:
embeddings = OpenAIEmbeddings()

### Store embeddings in SingleStoreDB

In [22]:
docsearch = SingleStoreDB.from_documents(
    texts,
    embeddings,
    table_name="rag_store",  # Table name to store chunks
    host="svc-98ef7484-559e-498d-b400-efdec97e2064-dml.aws-london-1.svc.singlestore.com",  # Replace with your SingleStore host
    port=3306,
    user="admin",  # Replace with your SingleStore username
    password="ih4HN1zgS1cilvRLz1ZT51m7WulIVbYU",  # Replace with your SingleStore password
    database="LlamaDB"  # Replace with your SingleStore database name
)

### Initialize the Llama 4 model via OpenRouter

In [23]:
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="Add Your OpenRouter API Key",  # Replace with your OpenRouter API key
    model="meta-llama/llama-4-maverick:free",
    temperature=0.7,
    max_tokens=4096
)

### Create a RetrievalQA chain

In [24]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

### Ask a question about your PDF content

In [25]:
query = "What environmental trends in India are highlighted in the EnviStats India 2024 report?"
response = qa_chain.run(query)
print("Response:", response)

/tmp/ipykernel_75/1583438535.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Response: The EnviStats India 2024 report highlights the following environmental trends and concerns in India:

1. Concerns about rapidly depleting vital resources.
2. Adverse impacts on the natural environment.
3. Land degradation.
4. Biodiversity loss.
5. Climate change.

The report also mentions that India has set a target of achieving net zero carbon emission by 2070 and is working towards a circular economy through a combination of conservation and efficiency measures. 

Additionally, it is mentioned that India has a rich biodiversity wealth and natural resources that are closely interlinked with the lives and livelihoods of the people, especially in rural and remote areas, and that there is a cultural emphasis on living in harmony with nature.


In [27]:
query = "Summarize the key findings on air pollution from the EnviStats India 2024 report."
response = qa_chain.run(query)
print("Response:", response)

Response: I don't know. The provided context does not contain information about the key findings on air pollution from the EnviStats India 2024 report. The given text appears to be introductory sections and page numbers of the report, but it doesn't provide specific data or findings on air pollution.


In [28]:
query = "Can you name the team of officers associated with the publication?"
response = qa_chain.run(query)
print("Response:", response)

Response: The team of officers associated with the publication includes:

1. Shri N.K. Santoshi - Director General
2. Shri Subash Chandra Malik - Additional Director General
3. Dr. Sanjay Kumar - Deputy Director General
4. Dr. Sudeepta Ghosh - Director
5. Ms. Kirti Nandkishor Gaikwad - Joint Director
6. Dr. Ruchi Mishra - Deputy Director
7. Ms. Kulpreet Sokhi - Senior Statistical Officer
8. Ms. Dipika Gupta - Junior Statistical Officer
9. Md Sahil Alam - Stenographer D

These officers are listed at the beginning of the document.


In [29]:
query = "What are they talking about in chapter4 in the publication?"
response = qa_chain.run(query)
print("Response:", response)

Response: According to the index and the text provided, Chapter 4 of the publication "EnviStats India 2024: Environment Accounts" is about the "Soil Nutrient Index". Specifically, it provides information on the soil nutrient index for the period 2023-24, using data collected for preparing Soil Health Cards.


In [30]:
query = "According to SEEA-Energy, what are the are the three types of additions to the stock of the Energy Assets?"
response = qa_chain.run(query)
print("Response:", response)

Response: According to SEEA-Energy, the three types of additions to the stock of the Energy Assets are:

1. Discoveries: New deposits found during an accounting period.
2. Upward reappraisals: Additions in the estimated available stock of a specific deposit or changes in categorization of specific deposits based on new geological information, technology, or resource price.
3. Reclassifications: Changes in the classification of deposits due to government decisions concerning access rights to a deposit.


In [33]:
query = "What is the Figure-2.5 is all about?"
response = qa_chain.run(query)
print("Response:", response)

Response: According to the text, Figure 2.5 shows the supply of Energy in Peta Joules over the years, indicating a generally increasing trend.


In [34]:
query = "What is the Figure-2.6 is all about?"
response = qa_chain.run(query)
print("Response:", response)

Response: According to the given text, Figure 2.6 shows the industry-wise use of energy for the year 2022-23. The highest share of use can be seen in the "Other industries", "Manufacturing", and the "Electricity sector" for that year.


In [37]:
query = "What is the Figure 4.2 is all about?"
response = qa_chain.run(query)
print("Response:", response)

Response: According to the context provided, Figure 4.2 is about "Soil Distribution in India".
